In [ ]:
from zizou.data import DataSource, GeoMagWaveforms
from zizou.ssam import SSAM 
from datetime import datetime, date, timedelta, timezone
import pandas as pd
import requests
from tonik import Storage
from io import StringIO
from obspy import UTCDateTime, Trace
import numpy as np
from zizou.visualise import plot_ssam_plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
gmc = GeoMagWaveforms(base_url='https://tilde.geonet.org.nz/v3/data',
                      method='60s', aspect='F-total-field', name='magnetic-field')
ssam = SSAM(per_lap=0.5, timestamp='start', interval=1024*60.,
            frequencies=np.linspace(1/1000., 1/120., 50))
ds = DataSource(clients=[gmc])
sg = Storage('geomag', rootdir='/tmp/geomag')
st = sg.get_substore(site='EYWM', sensor='50', channel='F')
startdate = UTCDateTime(2024, 4, 1)
enddate = UTCDateTime.now()
enddate = UTCDateTime(2024, 4, 9)
for tr in ds.get_waveforms(net='NZ', site='EYWM', loc='50', comp='F',
                           start=startdate, end=enddate,
                           cache=True):
    xds = ssam.compute(tr) 
    st.save(xds)

In [ ]:
st.starttime = startdate.datetime
st.endtime = enddate.datetime
xds_st = st('filterbank')

In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
tr = next(ds.get_waveforms(net='NZ', site='EYWM', loc='50', comp='F',
                           start=startdate, end=enddate))
dt = [startdate.datetime + timedelta(seconds=s) for s in tr.times()]
fig.add_trace(go.Scatter(x=dt, y=tr.data, mode='lines'), row=1, col=1)
fig.add_trace(plot_ssam_plotly(xds_st, dbscale=True, new_fig=False), row=2, col=1)